## Notebook overview:
- In this notebook we load a csv files with links of pictures and tag them
- First we install the TFOD API
- Then we load the csv file out of csv directory into a data frame and manipulate them for our needs
- After that we create a list out of our Panda Series with links
- Then we need to define a function for loading the pictures from an url
- We predict each url(picture) and write the prediction to the dataframe
- Finally we can save the datframe in the csv format

In [ ]:
#connect to our drive
from google.colab import drive
drive.mount('/content/drive')

## Cloning TFOD 2.0 Github needed for inferencing

In [ ]:
cd /content/

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 68944, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 68944 (delta 50), reused 74 (delta 42), pack-reused 68860
Receiving objects: 100% (68944/68944), 577.22 MiB | 52.59 MiB/s, done.
Resolving deltas: 100% (48555/48555), done.


In [ ]:
cd /content/models/research

/content/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

In [ ]:
cd cocoapi/PythonAPI

/content/models/research/cocoapi/PythonAPI


In [ ]:
!make

### Install the Object Detection API

In [ ]:
cd /content/models/research

/content/models/research


In [ ]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: pymongo
    Found existing installation: pymongo 4.0.1
    Uninstalling pymongo-4.0.1:
      Successfully uninstalled pymongo-4.0.1
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.

In [ ]:
# From within TensorFlow/models/research/
#test if tensorflow downloaded successfully
!python object_detection/builders/model_builder_tf2_test.py

2022-02-09 12:06:56.369141: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0209 12:06:56.743719 139629034796928 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.64s
I0209 12:06:57.008351 139629034796928 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.64s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.52s
I0209 12:06:57.528903 139629034796928 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_cr

### Import csv file, add needed columns and create a url_list 

In [ ]:
cd /content/drive/MyDrive/Human Detection Tchibo #path to  csv file

/content/drive/MyDrive/Human Detection Tchibo


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df= pd.read_csv("tchibo_community_images.csv", header = None, names=["link", "user"] )

In [ ]:
df["detection_class"] = np.nan
df["confidence_score"] = np.nan

In [ ]:
df.head()

link  ... confidence_score
0  https://assets-tch-live.s3.eu-central-1.amazon...  ...              NaN
1  https://assets-tch-live.s3.eu-central-1.amazon...  ...              NaN
2  https://assets-tch-live.s3.eu-central-1.amazon...  ...              NaN
3  https://assets-tch-live.s3.eu-central-1.amazon...  ...              NaN
4  https://assets-tch-live.s3.eu-central-1.amazon...  ...              NaN

[5 rows x 4 columns]

In [ ]:
df.user.value_counts()

2e95251e28153ff771579f7c2330368d    7
badc9b7866e336a9ae6e0db58f4ba7d1    5
6845c7f8c91ce1f985fecfc39a03c33a    5
67b21469e20c03ba49af6d78bffb46fc    5
e9773d68cc2975c1dfca9b9e9fa2a871    4
                                   ..
3076d12136e4a3a18271f7b13110f39c    1
2e95a3d662ed51e6a2aa7917419c0825    1
0be8cf3c99316e2284ac832681beeaf9    1
d2d589ab7b56c0cb5ce072821747fc7f    1
a086e4344da34ad99e90426d61827160    1
Name: user, Length: 299, dtype: int64

In [ ]:
url_list= df[0].tolist()

--> hier können wir dann noch eine kleine explorative datenanalyse machen. wer postet am meisten Tchibofotos ... und wer welche mit hohen confidence score (gut zum Marketing verwenden) --> groupby...

##Load the model

In [ ]:
!pip uninstall numpy
!pip install numpy --upgrade

In [ ]:
#upgrade specific opencv liabaries to have all opencv libaries up to date
!pip uninstall opencv-python
!pip install opencv-python
!pip uninstall opencv-contrib-python
!pip install opencv-contrib-python
!pip uninstall opencv-python-headless
!pip install opencv-python-headless

In [ ]:
# load the model
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

path_to_saved_model = "/content/drive/MyDrive/training_demo/exported_models/CenterNet_1024x1024_exported/saved_model/saved_model"

print('Loading model...', end='')
start_time = time.time()

# load saved model 
detect_fn = tf.saved_model.load(path_to_saved_model)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

## Predict each url and write the first detection class and belonging confidence score to dataframe with urls

In [ ]:
# LOAD LABEL MAP DATA FOR PLOTTING
# PROVIDE PATH TO LABEL MAp

path_to_labels = '/content/drive/MyDrive/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels,
                                                                    use_display_name=True)
#import all packages
import cv2 
from google.colab.patches import cv2_imshow                                                                   
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
from skimage import io
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

#define function reading image from url 
def load_image_from_url(url):
  return io.imread(url)



#iterate through url list and display each detection
i=0
for urls in url_list:

  image= load_image_from_url(urls) #--> that does not work yet
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  #image_expanded = np.expand_dims(image_rgb, axis=0)

  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]

  # input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  #write detections to a dataframe
  df.iloc[i, 2]= detections['detection_classes'][0]
  df.iloc[i, 3]= detections['detection_scores'][0]

  i+=1

Display the detection class and confidence scroe to compare to dataframe data. 
Checks if data was written correctly to dataframe

In [ ]:
# LOAD LABEL MAP DATA FOR PLOTTING
# PROVIDE PATH TO LABEL MAP
path_to_labels = '/content/drive/MyDrive/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels,
                                                                    use_display_name=True)
#import all packages
import cv2 
from google.colab.patches import cv2_imshow                                                                   
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
from skimage import io
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

#define function reading image from url 
def load_image_from_url(url):
  return io.imread(url)

#iterate through image path list and display each detection
i=0
for urls in url_list:
  if i >4:
    break
  i+=1
  image= load_image_from_url(urls) #--> that does not work yet
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]

  # input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  image_with_detections = image.copy()

  # set min_score_thresh based on your minimum thrshold for detection 
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.25,
        agnostic_mode=False)

  # display output image 
  cv2_imshow(image_with_detections)
  

In [ ]:
df.head()

link  ... confidence_score
0  https://assets-tch-live.s3.eu-central-1.amazon...  ...         0.153515
1  https://assets-tch-live.s3.eu-central-1.amazon...  ...         0.446764
2  https://assets-tch-live.s3.eu-central-1.amazon...  ...         0.039604
3  https://assets-tch-live.s3.eu-central-1.amazon...  ...         0.041130
4  https://assets-tch-live.s3.eu-central-1.amazon...  ...         0.069211

[5 rows x 4 columns]

### Write dataframe to csv and download csv